In [473]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import tensorflow as tf
import numpy as np

In [474]:
df_sessions = pd.read_json(path_or_buf='../../data/raw/sessions.jsonl', lines=True)
df_products = pd.read_json(path_or_buf='../../data/raw/products.jsonl', lines=True)
df_users = pd.read_json(path_or_buf='../../data/raw/users.jsonl', lines=True)

In [475]:
df_users.head()

,user_id,name,city,street
0,102,Aurelia Malon,Poznań,pl. Brzoskwiniowa 11/53
1,103,Mateusz Kobel,Poznań,al. Wrocławska 10
2,104,Radosław Ratka,Szczecin,pl. Nowa 89/04
3,105,Anastazja Oszust,Warszawa,ul. Częstochowska 80
4,106,Sylwia Nurek,Warszawa,al. Wiosenna 72


In [476]:
df_products.head()

,product_id,product_name,category_path,price
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
1,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,2048.50
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99


In [477]:
categories_one_hot = df_products.category_path.str.get_dummies(';')
categories_one_hot.head()

,Akcesoria telefoniczne,Anteny RTV,Audio,Biurowe urządzenia wielofunkcyjne,Drukarki i skanery,Gry PlayStation3,Gry Xbox 360,Gry i konsole,Gry komputerowe,Gry na konsole,...,Słuchawki,Tablety,Tablety i akcesoria,Telefony i akcesoria,Telefony komórkowe,Telefony stacjonarne,Telewizory i akcesoria,Video,Zestawy głośnomówiące,Zestawy słuchawkowe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [478]:
cities_one_hot = df_users.city.str.get_dummies()
cities_one_hot.head()

,Gdynia,Kraków,Poznań,Radom,Szczecin,Warszawa,Wrocław
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0


In [479]:
df_products.drop(columns=['product_name', 'category_path'], inplace=True)
df_products = df_products.join(categories_one_hot)

In [480]:
df_users.drop(columns=['city'], inplace=True)
df_users = df_users.join(cities_one_hot)

In [481]:
df_products.head()

,product_id,price,Akcesoria telefoniczne,Anteny RTV,Audio,Biurowe urządzenia wielofunkcyjne,Drukarki i skanery,Gry PlayStation3,Gry Xbox 360,Gry i konsole,...,Słuchawki,Tablety,Tablety i akcesoria,Telefony i akcesoria,Telefony komórkowe,Telefony stacjonarne,Telewizory i akcesoria,Video,Zestawy głośnomówiące,Zestawy słuchawkowe
0,1001,58.97,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1002,2048.50,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1003,7639.00,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1004,49.99,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1005,49.99,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [482]:
df_sessions.head()

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,124,2021-07-28 15:47:21,102,1283,VIEW_PRODUCT,5,NaN
1,124,2021-07-28 15:47:40,102,1283,BUY_PRODUCT,5,20001.0
2,125,2021-09-12 16:43:16,102,1076,VIEW_PRODUCT,0,NaN
3,125,2021-09-12 16:46:24,102,1080,VIEW_PRODUCT,0,NaN
4,125,2021-09-12 16:48:08,102,1079,VIEW_PRODUCT,0,NaN


In [483]:
df_users['sex'] = ['Female' if name.split(' ')[0][-1] == 'a' else 'Male' for name in df_users['name']]
sex_one_hot = df_users.sex.str.get_dummies()
df_users.drop(columns=['sex'], inplace=True)
df_users = df_users.join(sex_one_hot)

In [484]:
df_users.head()

,user_id,name,street,Gdynia,Kraków,Poznań,Radom,Szczecin,Warszawa,Wrocław,Female,Male
0,102,Aurelia Malon,pl. Brzoskwiniowa 11/53,0,0,1,0,0,0,0,1,0
1,103,Mateusz Kobel,al. Wrocławska 10,0,0,1,0,0,0,0,0,1
2,104,Radosław Ratka,pl. Nowa 89/04,0,0,0,0,1,0,0,0,1
3,105,Anastazja Oszust,ul. Częstochowska 80,0,0,0,0,0,1,0,1,0
4,106,Sylwia Nurek,al. Wiosenna 72,0,0,0,0,0,1,0,1,0


In [485]:
df = df_sessions.merge(df_products, left_on='product_id', right_on='product_id')
df = df.merge(df_users, left_on='user_id', right_on='user_id')
df.head()

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,price,Akcesoria telefoniczne,Anteny RTV,...,street,Gdynia,Kraków,Poznań,Radom,Szczecin,Warszawa,Wrocław,Female,Male
0,124,2021-07-28 15:47:21,102,1283,VIEW_PRODUCT,5,NaN,99.99,0,0,...,pl. Brzoskwiniowa 11/53,0,0,1,0,0,0,0,1,0
1,124,2021-07-28 15:47:40,102,1283,BUY_PRODUCT,5,20001.0,99.99,0,0,...,pl. Brzoskwiniowa 11/53,0,0,1,0,0,0,0,1,0
2,139,2021-07-01 11:28:02,102,1283,VIEW_PRODUCT,0,NaN,99.99,0,0,...,pl. Brzoskwiniowa 11/53,0,0,1,0,0,0,0,1,0
3,139,2021-07-01 11:31:18,102,1283,BUY_PRODUCT,0,20013.0,99.99,0,0,...,pl. Brzoskwiniowa 11/53,0,0,1,0,0,0,0,1,0
4,144,2021-03-19 13:55:04,102,1283,VIEW_PRODUCT,0,NaN,99.99,0,0,...,pl. Brzoskwiniowa 11/53,0,0,1,0,0,0,0,1,0


In [486]:
aggregation_functions = {
    'timestamp' : lambda t: [(t.max() - t.min()).seconds, t.min(), t.max()],
    'user_id' : 'first',
    'product_id' : 'unique',
    'event_type' : lambda e: 1 if len(e.unique()) > 1 else 0,
    'offered_discount' : 'first',
    'price' : lambda p: p.sum()
}

for name in categories_one_hot.columns:
    aggregation_functions[name] = lambda n: n.sum()

# for name in cities_one_hot.columns:
#     aggregation_functions[name] = 'first'

for name in sex_one_hot.columns:
    aggregation_functions[name] = 'first'

main_df = df.groupby(df['session_id']).aggregate(aggregation_functions)
main_df.rename(columns={'event_type' : 'purchased'}, inplace=True)

In [487]:
timestamp_dict = [{'session_length' : x[0], 'session_start' : x[1], 'session_end' : x[2]} for x in main_df['timestamp']]
timestamp_df = pd.DataFrame(timestamp_dict)

In [488]:
main_df = pd.concat([main_df, timestamp_df], axis=1, join="inner")
main_df.drop(columns=['timestamp'], inplace=True)
main_df.index.name = 'session_id'
main_df = main_df[['user_id', 'product_id', 'offered_discount', 'session_length', 'price', 'session_start', 'session_end', 'purchased'] + list(sex_one_hot.columns.values) + list(categories_one_hot.columns.values)]
main_df.head()

,user_id,product_id,offered_discount,session_length,price,session_start,session_end,purchased,Female,Male,...,Słuchawki,Tablety,Tablety i akcesoria,Telefony i akcesoria,Telefony komórkowe,Telefony stacjonarne,Telewizory i akcesoria,Video,Zestawy głośnomówiące,Zestawy słuchawkowe
session_id,,,,,,,,,,,,,,,,,,,,,
124,102,[1283],5,725,199.98,2021-08-26 22:27:52,2021-08-26 22:39:57,1,1,0,...,0,0,0,0,0,0,2,2,0,0
125,102,"[1076, 1080, 1079, 1003, 1002, 1075, 1078, 1077]",0,1547,36664.54,2021-03-08 08:14:24,2021-03-08 08:40:11,1,1,0,...,0,0,0,0,0,0,0,0,0,0
126,102,"[1281, 1278]",15,347,222.60,2021-02-16 17:40:31,2021-02-16 17:46:18,1,1,0,...,0,0,0,0,0,0,0,0,0,0
127,102,"[1281, 1278]",0,457,208.50,2021-03-13 06:48:13,2021-03-13 06:55:50,1,1,0,...,0,0,0,0,0,0,0,0,0,0
128,102,"[1008, 1007, 1013, 1012, 1004, 1010]",0,1010,568.94,2021-04-18 21:02:01,2021-04-18 21:18:51,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [489]:
main_df['n_views'] = [len(x) for x in main_df['product_id'].values]

In [490]:
main_df.head()

,user_id,product_id,offered_discount,session_length,price,session_start,session_end,purchased,Female,Male,...,Tablety,Tablety i akcesoria,Telefony i akcesoria,Telefony komórkowe,Telefony stacjonarne,Telewizory i akcesoria,Video,Zestawy głośnomówiące,Zestawy słuchawkowe,n_views
session_id,,,,,,,,,,,,,,,,,,,,,
124,102,[1283],5,725,199.98,2021-08-26 22:27:52,2021-08-26 22:39:57,1,1,0,...,0,0,0,0,0,2,2,0,0,1
125,102,"[1076, 1080, 1079, 1003, 1002, 1075, 1078, 1077]",0,1547,36664.54,2021-03-08 08:14:24,2021-03-08 08:40:11,1,1,0,...,0,0,0,0,0,0,0,0,0,8
126,102,"[1281, 1278]",15,347,222.60,2021-02-16 17:40:31,2021-02-16 17:46:18,1,1,0,...,0,0,0,0,0,0,0,0,0,2
127,102,"[1281, 1278]",0,457,208.50,2021-03-13 06:48:13,2021-03-13 06:55:50,1,1,0,...,0,0,0,0,0,0,0,0,0,2
128,102,"[1008, 1007, 1013, 1012, 1004, 1010]",0,1010,568.94,2021-04-18 21:02:01,2021-04-18 21:18:51,1,1,0,...,0,0,0,0,0,0,0,0,0,6


## Przygotowanie danych do modelu

In [491]:
main_df.reset_index(inplace=True)

In [492]:
main_df.drop(columns=['session_id', 'user_id', 'product_id'], inplace=True)

In [493]:
dates_encoded = pd.DataFrame({
    "month": main_df['session_start'].dt.month,
    "day": main_df['session_start'].dt.day,
    "hour": main_df['session_start'].dt.hour,
    "dayofweek": main_df['session_start'].dt.dayofweek,
})

In [494]:
main_df = main_df.join(dates_encoded)

In [495]:
main_df.drop(columns=['session_start', 'session_end'], inplace=True)

In [496]:
main_df.head()

,offered_discount,session_length,price,purchased,Female,Male,Akcesoria telefoniczne,Anteny RTV,Audio,Biurowe urządzenia wielofunkcyjne,...,Telefony stacjonarne,Telewizory i akcesoria,Video,Zestawy głośnomówiące,Zestawy słuchawkowe,n_views,month,day,hour,dayofweek
0,5,725,199.98,1,1,0,0,0,0,0,...,0,2,2,0,0,1,8,26,22,3
1,0,1547,36664.54,1,1,0,0,0,0,9,...,0,0,0,0,0,8,3,8,8,0
2,15,347,222.60,1,1,0,0,0,0,0,...,0,0,0,0,0,2,2,16,17,1
3,0,457,208.50,1,1,0,0,0,0,0,...,0,0,0,0,0,2,3,13,6,5
4,0,1010,568.94,1,1,0,0,0,0,0,...,0,0,0,0,0,6,4,18,21,6


# Wydzielenie zbioru treningowego, walidacyjnego i testowego

In [497]:
y = main_df['purchased'].values
X = main_df.drop(columns=['purchased'])

In [498]:
X

,offered_discount,session_length,price,Female,Male,Akcesoria telefoniczne,Anteny RTV,Audio,Biurowe urządzenia wielofunkcyjne,Drukarki i skanery,...,Telefony stacjonarne,Telewizory i akcesoria,Video,Zestawy głośnomówiące,Zestawy słuchawkowe,n_views,month,day,hour,dayofweek
0,5,725,199.98,1,0,0,0,0,0,0,...,0,2,2,0,0,1,8,26,22,3
1,0,1547,36664.54,1,0,0,0,0,9,9,...,0,0,0,0,0,8,3,8,8,0
2,15,347,222.60,1,0,0,0,0,0,0,...,0,0,0,0,0,2,2,16,17,1
3,0,457,208.50,1,0,0,0,0,0,0,...,0,0,0,0,0,2,3,13,6,5
4,0,1010,568.94,1,0,0,0,0,0,0,...,0,0,0,0,0,6,4,18,21,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9398,5,0,2677.90,0,1,4,0,0,0,0,...,0,0,0,0,4,3,1,27,3,2
9399,0,116,31328.54,0,1,0,0,0,8,8,...,0,0,0,0,0,7,7,6,16,1
9400,5,1617,1121.66,0,1,0,10,0,0,0,...,0,10,10,0,0,9,3,7,7,6
9401,0,0,17031.50,0,1,0,0,0,4,4,...,0,0,0,0,0,4,10,12,0,1


In [499]:
y

array([1, 1, 1, ..., 1, 0, 1])

In [500]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [501]:
X_test.shape

(2821, 38)

In [502]:
X_train.shape

(6582, 38)

## Szukanie modelu

#### Regresja liniowa

In [503]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

LinearRegression()

In [504]:
predictions = linear_regression.predict(X_test) > 0.5
accuracy_score(y_test, predictions)

1.0

## RandomForest

In [505]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

RandomForestClassifier()

In [506]:
rand_forest_scores = cross_val_score(random_forest, X_train, y_train, cv=10, scoring="accuracy")
rand_forest_scores.mean()

0.9688558237358805

## KNN

In [507]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [508]:
knn_forest_scores = cross_val_score(knn, X_train, y_train, cv=10, scoring="accuracy")
knn_forest_scores.mean()

0.7260736309504592

## SVM

In [509]:
svm = SVC(kernel="rbf")

In [510]:
svm_scores = cross_val_score(svm, X_train, y_train, cv=10, scoring="accuracy")
svm_scores.mean()

0.5850803234153249

## Gradient Boosting

In [511]:
gradient_boosting = GradientBoostingClassifier()

In [512]:
gradient_boosting_scores = cross_val_score(gradient_boosting, X_train, y_train, cv=3, scoring="accuracy")
gradient_boosting_scores.mean()

0.9152233363719234

## Sieć neuronowa

In [513]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=len(main_df.columns)-1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")

])

In [514]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 128)               4992      
                                                                 
 dense_68 (Dense)            (None, 64)                8256      
                                                                 
 dense_69 (Dense)            (None, 32)                2080      
                                                                 
 dense_70 (Dense)            (None, 1)                 33        
                                                                 
Total params: 15,361
Trainable params: 15,361
Non-trainable params: 0
_________________________________________________________________


In [515]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [516]:
model.fit(np.array(X_train,dtype=np.float64), np.array(y_train,dtype=np.float64), epochs=200)

Epoch 1/200
206/206 [==============================] - 0s 578us/step - loss: 10.4813 - accuracy: 0.5258
Epoch 2/200
206/206 [==============================] - 0s 660us/step - loss: 5.3582 - accuracy: 0.5460
Epoch 3/200
206/206 [==============================] - 0s 619us/step - loss: 4.1893 - accuracy: 0.5684
Epoch 4/200
206/206 [==============================] - 0s 641us/step - loss: 2.7027 - accuracy: 0.5869
Epoch 5/200
206/206 [==============================] - 0s 634us/step - loss: 2.8902 - accuracy: 0.5805
Epoch 6/200
206/206 [==============================] - 0s 619us/step - loss: 2.3743 - accuracy: 0.5954
Epoch 7/200
206/206 [==============================] - 0s 583us/step - loss: 1.9626 - accuracy: 0.6026
Epoch 8/200
206/206 [==============================] - 0s 648us/step - loss: 1.0731 - accuracy: 0.6275
Epoch 9/200
206/206 [==============================] - 0s 603us/step - loss: 1.0177 - accuracy: 0.6235
Epoch 10/200
206/206 [==============================] - 0s 590us/step - 

In [517]:
predictions = model.predict(np.array(X_test,dtype=np.float64)) > 0.5

In [518]:
accuracy_score(y_test, predictions)

0.9113789436370081